# Hometask 02
### Maksim Sorokin

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных:
- Привести колонки датафрейма к одному типу данных
- Проверить индекс, чтобы он соответствовал порядку строк
- Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки

In [54]:
import pandas as pd

In [55]:
df = pd.read_excel('output/dom_objects.xlsx')


Для исследования в задании 3 необходимы следюущие столбцы из "сырого" массива загруженных из api данных,
поэтому типы данных и пропуски данных буду смотреть только в тех столбцах, что представляют интерес в целях исследования
- регион
- средняя цена за квадратный метр
- объем вводимого жилья
- год ввода в эксплуатацию
- этажность
- количество квартир
- жилая площадь 
- количество парковочных мест

In [62]:
assert df.region.dtype == 'int64'           # регион
assert df.objPriceAvg.dtype == 'float64'    # средняя цена за квадратный метр
## здесь лежит object, его надо конвертировать
df.objReady100PercDt = pd.to_datetime(df.objReady100PercDt, format='%Y-%m-%d')
assert df.objSquareLiving.dtype == 'float64'    # жилая площадь (самое подходящее поле для "Объема вводимого жилья")
assert df.floorMax.dtype == 'float64'                 # этажность
assert df.objReady100PercDt.dtype == 'datetime64[ns]' # дата ввода в эксплуатацию
assert df.objFlatCnt.dtype == 'int64'       # количество квартир
assert df.objElemParkingCnt.dtype =='int64' # объем парковочных мест

columns_for_research = set()
columns_for_research.update(['region','objPriceAvg','objReady100PercDt', 'objSquareLiving', 'floorMax', 'objFlatCnt', 'objElemParkingCng'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10660 entries, 0 to 10659
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Unnamed: 0                    10660 non-null  int64         
 1   id                            10660 non-null  int64         
 2   pdId                          10660 non-null  int64         
 3   region                        10660 non-null  int64         
 4   address                       10634 non-null  object        
 5   objElemLivingCnt              10660 non-null  int64         
 6   objReady100PercDt             10660 non-null  datetime64[ns]
 7   wallMaterialShortDesc         10660 non-null  object        
 8   objLkFinishTypeDesc           10660 non-null  object        
 9   objLkFreePlanDesc             10648 non-null  object        
 10  objElemParkingCnt             10660 non-null  int64         
 11  objSquareLiving             

Отбрасываю неинтересующую меня информацию, проверяю индекс

In [61]:
df2 = df
for a in df.columns:
    if a  not in columns_for_research: 
        df2 = df2.drop(labels=a, axis=1)


assert df2.index.is_unique
df2.shape
df2.head(3)
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10660 entries, 0 to 10659
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   region             10660 non-null  int64         
 1   objReady100PercDt  10660 non-null  datetime64[ns]
 2   objSquareLiving    10660 non-null  float64       
 3   objFlatCnt         10660 non-null  int64         
 4   floorMax           10659 non-null  float64       
 5   objPriceAvg        7608 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 499.8 KB


Видим, большой датафрейм с полным набором занимал 5.4+ MB, а датафрейм с только интересующими нас данными - 499.8 KB
Так же по выводу info видимо, что пустые значения есть в столбцах floorMax & objPriceAvg, считаю их:

In [97]:
df2.isnull().sum()

region                  0
objReady100PercDt       0
objSquareLiving         0
objFlatCnt              0
floorMax                1
objPriceAvg          3052
dtype: int64

Заменяю единственный пропуск в этажности на среднее значение, контролирую результат

In [116]:

df_fill_na_floor = df2.fillna({'floorMax': df2.floorMax.mean()})
df_fill_na_floor.isnull().sum()

region                  0
objReady100PercDt       0
objSquareLiving         0
objFlatCnt              0
floorMax                0
objPriceAvg          3052
dtype: int64

А вот с ценой - вычищаю эти строки совсем, проверяю результат

In [118]:
final_df = df_fill_na_floor.dropna(how='any')
final_df.shape

(7608, 6)

Сохраняю рафинированный датафрейм для исследования в рамках следующего задания

In [120]:
final_df.to_pickle("output/dom_refined.pkl")